# Fundamentals of Computer Science 30398 - Lecture 7

### Exercise 1 - Queens problem

Write a function `queens(n)` that takes as an argument a size of a chessboard $n$, and returns in how many ways it is possible to put $n$ pairwise non-attacking queens on $n\times n$ chessboard.

Two queens are attacking each other, if they are on the same row, same column or the same diagonal.

We will first write a short function that checks whether two queens are attacking each other - surely, such a function will be useful at some point, and it is easy to write. To this end, we need to decide how we will represent a queens position throughout our program. We chose to keep it as a pair of `int` variables, each between $0$ and $n-1$ - the first element of the pair is the index of the row, and the second the index of the column where the potential queen is placed.

As mentioned above: two queens are attaching each other if they are on the same row, same column or the same diagonal. It is very easy to check if two queens are on the same row or the same column - we compare if the first (resp. second) element of each pair is the same.

To check if two queens are on the same diagonal, we note that the diagonals of the chessboard have constant value of the sum (or the difference) of coordinates.

In [16]:
def is_attacking(q1, q2):
    same_row_or_column = (q1[0] == q2[0]) or (q1[1] == q2[1])
    same_diagonal = (q1[1] - q1[0] == q2[1] - q2[0]) or (q1[1] + q1[0] == q2[1] + q2[0])
    return same_row_or_column or same_diagonal

Let us check if this works on a few simple examples:

In [17]:
is_attacking( (0, 0), (3, 3) ) == True

True

In [18]:
is_attacking( (0, 1), (3, 3) ) == False

True

Let us now write the main function that calculates the number of valid positions of $n$ queens on an $n\times n$ chess board. On a first glance, it is not obvious how such a function should work - we have learned about recursion, but there is no obvious recursive structure here: even if we knot all valid configurations of $n-1$ queens on $n-1\times n-1$ chessboard, this doesn't help us to solve the problem on $n\times n$ chess board.

The key idea is to **generalize** the problem. Note first that in every configuration of non-attacking queens, clearly there is at most one queen in each row. Since we are attempting to count positions of $n$ queens on $n \times n$ chessboard, every valid configuration has exactly one queen on each row.

We will use this observation, and write a function that takes the size of the chessboard $n$, and a list `positions`, of some $k$ non-attacking queens in the first $k$ rows; the function is supposed to count in how many ways we can extend those $k$ queens to a full valid configuration of $n$ queens (where a position is "valid" if none of the queens is attacking any other).

If we could solve this problem, we can clearly solve the original problem: calling `queens(n, [])` asks in how many ways we can extend an empty lists of queens to the full configuration - which is exactly the number of placements of $n$ queens on $n\times n$ chessboard. 

The benefit of stating a problem this way is that it now has a very clear recursive structure: to calculate in how many ways we can extend a given list `poistions` to a full configuration, we first check if the number of queens already placed is $n$ --- in which case there is only one "extension" -- the position is already valid, and we do not need to do anything.

If the number of queens already placed $k$ is strictly smaller than $n$, we try to place the $k+1$-st queen in each of $n$ possible locations. For a potential placement $(k,i)$ of the last queen, we first check if it is not attacking any of the previous queens in the list, if it is not --- we append its position it to the list `position` and call the function `queens(n, positions + [(k,i)])` - recursively calculating in how many ways this list can be extended to a full configuration. We sum those outputs over all potential positions for the $k+1$-st queen, and return the result as a number of valid extensions of the given list `position`.

At this stage, when iterating over all posisble positions $i$ for the $k+1$-st queen using the `for` loop, we realize that we would like to check whether the queen at position $(k, i)$ is attacking any of the already placed queens in the list `positions`. Since this is a relatively self-contained task, that would add some complexity, let us not worry about it for now: let's pretend that a function `is_valid_extension` is already written, and use call this function in the code below. We will implement this function a bit later.

By ignoring the implementation of the function `is_valid_extension` for now, the implementation of the main function itself is relatively simple recursion, not too much different than the previous recursive function we wrote to list all possible subsets of the set $\{0, \ldots, n-1\}$.

In [41]:
def queens(n, position):
    if len(position) == n:
        return 1
    k = len(position)
    result = 0
    for i in range(n):
        if is_valid_extension(position, (k, i)):
            result += queens(n, position + [(k, i)])
    return result

Now, we are left with writing a function that takes a list of $k$ positions of non-attacking queens, and another position `new_queen` --- the function is supposed to check if the `new_queen` is attacked by any of the previous queens on the list and return `False` in this case --- since this is not a valid extension. If the new queen is not attacked by any of the previous queens, we should return `True`.

Assuming that we already have a function `is_attacking` in place that checks whether a given pair of queens is attacking each other, this function by itself again is relatively simple: we iterate over all queens in the list `position` and as soon as we find one which is attacking `new_queen` we return `False`. Otherwise we return `True`.

In [42]:
def is_valid_extension(position, new_queen):
    for queen in position:
        if is_attacking(new_queen, queen):
            return False
    return True

**Note** A word of caution, here and everywhere else in your code --- when writing those type of nested statements pay close attention to the indentation of respective statements --- in which block is the `return True` instruction? Where it should be? Compare this code with the following one:

```python
def is_valid_extension(position, new_queen):
    for queen in position:
        if is_attacking(new_queen, queen):
            return False
        return True
```

What is the difference between those two? Which one is correct?

We can look to see what are the answers for small values of `n`:

In [43]:
queens(1, position = [])

1

In [44]:
queens(2, position = [])

0

In [45]:
queens(4, position = [])

2

In [46]:
queens(8, position = [])

92

We can modify the function a bit to print every valid solution it found:

In [49]:
def queens_print(n, position):
    if len(position) == n:
        print(position)
        return 1
    k = len(position)
    result = 0
    for i in range(n):
        if is_valid_extension(position, (k, i)):
            result += queens_print(n, position + [(k, i)])
    return result

In [50]:
queens_print(4, [])

[(0, 1), (1, 3), (2, 0), (3, 2)]
[(0, 2), (1, 0), (2, 3), (3, 1)]


2

**Note** It is a bad style to copy-paste the queens function, creating a new function with almost identical code and different name. Instead, it would be better to add an (optional) parameter `print_solutions` to the original function `queens`, set by default to `False`, and print all the solutions if the value of this parameter is set to `True`. Try to modify the original code and do it.

# Merge sort

As a second part of this lecture, we will try to actually implement merge sort divide-and-conquer algorithm. The algorithm itself was covered already on the theoretical part of the course (Lecture 3, September 10, 2025 --- see the slides on Piazza). Back when we covered the algorithm, you learned it in the abstract, but not necessarily with enough programming skills to actually implement it. Now, we have learned all necessary tools.

The algorithm consists of two conceptually seperate parts: the divide-and-conquer strategy, and the `merge` function. At a high level, in order to sort a given list of $n$ elements, we split the array into two halves (or --- if the lenght is odd --- two parts length of which differ by one), recursively sort both of those lists, and then `merge` two sorted lists into one sorted list with the same content as both of them. Ignoring for now how to actually write the function `merge`, this recursive procedure is relatively simple to write.

## Assume that a function merge is already written:

In [ ]:
def merge(lst1, lst2):
    ...

`merge` takes two sorted lists, and merges their content into sorted list.

### Write the (recursive) function `merge_sort` using this `merge`.

In [26]:
def merge_sort(lst):
    if len(lst) == 0 or len(lst) == 1:
        return lst
    mid = len(lst) // 2
    left = merge_sort(lst[:mid])
    right = merge_sort(lst[mid:])
    return merge(left, right)

### Function `merge`

Function `merge` by itself is slightly more complex. We assume that the function is given two lists, `lst1` and `lst2`, both sorted. We would like to produce a new, sorted, list with the same content as both of those lists together. (Of course, we can't just concatenate those lists and sort them --- we are writing the sorting function just now!)

To this end, we keep two indices `left` and `right` pointing at some locations in `lst1` and `lst2` respectively (initially both zero), and a list `result` (initially empty), which we will use to accumulate the merged list. At a given step of the algorithm, the list `result` will be sorted, and will contain all elements from list `lst1` with indices smaller than `left`, and all elements from list `lst2` with indices smaller than `right`.

As long as both `left` and `right` indices are smaller than then lengths of respective lists, we would like to append the smaller of the two elements `lst1[left]` and `lst2[right]` to the end of the list `result`, and increase the respective index.

Finally, when one of the indices `left` or `right` reached the end of the list, we append all remaining elements from the other list to the list `result`.

(See more detailed, and visual explanation of the `merge` step of the mergesort algorithm on the slides from the theoretical part of the lecture.)

In [29]:
def merge(lst1, lst2):
    left, right = 0, 0
    result = []
    while left < len(lst1) and right < len(lst2):
        if lst1[left] < lst2[right]:
            result.append(lst1[left])
            left = left + 1
        else:
            result.append(lst2[right])
            right = right + 1
    for i in range(left, len(lst1)):
        result.append(lst1[i])
    for i in range(right, len(lst2)):
        result.append(lst2[i])
    return result

We can now try to see if it works:

In [52]:
merge([1,4,7,8,11,12], [2,3,4,7, 14, 15])

[1, 2, 3, 4, 4, 7, 7, 8, 11, 12, 14, 15]

Let's try to use the entire function `merge_sort` now:

In [53]:
merge_sort([4,7,2,1,7,89,23,5])

[1, 2, 4, 5, 7, 7, 23, 89]